Earlier, we talked about the importance of working with time-series data with a consistent sampling rate. In other words, it's easiest to work with data when all of the data points are evenly spaced in time.

Another challenge you will run into is that you will encounter ***signal noise*** that impacts the measurements in your time-series data. In this notebook, we will cover some potential causes of signal noise and ways of addressing them.

# Important: Run this code cell each time you start a new session!

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
!wget -Ncnp https://physionet.org/files/accelerometry-walk-climb-drive/1.0.0/raw_accelerometry_data/id00b70b13.csv

In [ ]:
df = pd.read_csv('id00b70b13.csv')

# Filter to only walking activity, which is given a code of 1
df = df[df['activity'] == 1]

# Process the time
df.rename(columns={'time_s': 'Time'}, inplace=True)
df = df[(df['Time']>=700) & (df['Time']<=710)]
df['Time'] = df['Time'] - df['Time'].min()

# Process the accel
df['Accel'] = np.sqrt(df['la_x']**2 + df['la_y']**2 + df['la_z']**2)*9.8

# Keep only crucial columns
keep_cols = ['Time', 'Accel']
df = df[keep_cols]
df.to_csv('walking.csv',index=False)

# What Causes Signal Noise

Sources of signal noise can be highly dependent upon the kind of data that is being collected. Below are three broad categories of signal noise:
* **Data entry:** People may manually enter information incorrectly into a survey or form, leading to outliers in your data (e.g., entering `600 bpm` instead of `60 bpm` for heart rate).
* **Intrinsic causes:** A sensor can exhibit unwanted random variation due to factors like electrical interference, thermal deviation, and electronic component limitations.
* **Extrinsic causes:** There may be disturbances outside of the sensor that get superimposed on our measurements. For instance, if we are trying to record someone's voice using a microphone, ambient background sounds can make it more difficult for us to analyze speech.

Unfortunately, there is no one-size-fits-all solution to identifying signal noise. Some cases of noise might be obvious, while others may be more subtle. There might also be situations where you think something that looks like noise is actually an important indicator of the phenomenon you are trying to measure. Making these determinations will require inspecting your data closely and reflecting on how it was collected, and picking the correct technique for addressing signal noise can be equally complex.

Below, we will cover two commonly-used techniques for dealing with sensor noise, but these are not comprehensive by any means.

# Outlier Detection

An ***outlier*** is a data point that is significantly different from the other data points in a dataset. Identifying and removing outliers can help ensure that your analyses do not get thrown off by random perturbations that do not mean anything.

To talk about different approaches to outlier detection, let's manually create a simple time-series of resting heart rate data, which we will assume should lie between 60–100 bpm.

In [ ]:
hr_data = [60, 66, 64, 73, 65, 200, 70, 64, 66, 60]
hr_time = list(range(len(hr_data)))
lower_bound = 60
upper_bound = 100

plt.figure(figsize=(5,3))
plt.plot(hr_time, hr_data, 'k-*')
plt.xlabel('Hour')
plt.ylabel('Heart Rate (bpm)')
plt.show()

There is a clear outlier at the 5th hour when a resting heart rate of 200 bpm was recorded. In fact, we can tell that this data point is an outlier without even knowing its timestamp, so the techniques we will cover below will primarily involve working with `hr_data` only.

## Identifying Outliers: Manual Method

If you know ahead of time that your data should fall within a specific range, you can use comparators and comparator-like functions to check if your data lies within that range. Below are three examples using `if` statements, `numpy`, and `pandas`:

In [ ]:
# Manual
is_outlier = []
for hr in hr_data:
    out_of_range = hr < lower_bound or hr > upper_bound
    is_outlier.append(out_of_range)
print(is_outlier)

In [ ]:
# Numpy boolean mask
hr_array = np.array(hr_data)
(hr_array < lower_bound) | (hr_array > upper_bound)

In [ ]:
# Pandas between
hr_series = pd.Series(hr_data)
~hr_series.between(lower_bound, upper_bound)

## Identifying Outliers: Z-Score

If you don't know the range in which your data should fall ahead of time, you can manually calculate the distribution of the data and then check to see how far away each point is from that distribution.

One way of doing this is by using the ***Z-score***, which measures how many standard deviations a data point is from the mean. When you use a measure like Z-score, you will need to decide how far away a data point should be in order for it to be considered an outlier. Let's look at the Z-score for each data point in our heart rate data:

In [ ]:
from scipy import stats
stats.zscore(hr_data)

Notice that most of our data points are within one standard deviation (i.e., their Z-score is between -1 and 1). However, our obviously incorrect data point has a Z-score that is close to 3. Therefore, we might consider that any data point with a Z-score greater than 2 or less than -2 should be considered an outlier in this case:

In [ ]:
np.abs(stats.zscore(hr_data)) > 2

## Handling Outliers: Placeholder Values


Handling outliers can be just as tricky as handling missing data since you don't know what the real value should have been at that timestamp.

One common approach you might have seen for handling outliers and missing data is by using a pre-determined ***placeholder value***.

Some common placeholder values you might see people use as they are saving data in Excel include `-1` (if you know that the data is always going to be positive) and `np.nan` (short for "not a number"). However, these values actually introduces new trends that make it more difficult to process your data later.

A safer placeholder value is one that you will know falls within the distribution of your data. For our heart rate data, for example, we might consider picking the average heart rate in our time series:

In [ ]:
# Compute the placeholder
hr_series = pd.Series(hr_data)
placeholder = hr_series.mean()

# Find the outlier entries and replace them
outlier_mask = ~hr_series.between(lower_bound, upper_bound)
hr_series[outlier_mask] = placeholder
print(hr_series)

## Handling Outliers: Interpolation

Placeholder values are applied uniformly across an entire dataset whenever there is an outlier. If your data has lots of outliers, then the same placeholder value will appear many times in your new signal. This may not be ideal since it will heavily skew the distribution of your measurements.

Rather than replacing all outliers using the same value, we can replace outliers with values that resemble nearby measurements. We can do this using the same interpolation techniques we explored earlier, bearing in mind that we should remove out outliers before generating our interpolator:

In [ ]:
# Convert to numpy
hr_time_arr = np.array(hr_time)
hr_data_arr = np.array(hr_data)

# Find the outlier entries and remove them
outlier_mask = (hr_array > lower_bound) & (hr_array < upper_bound)
hr_time_clean = hr_time_arr[outlier_mask]
hr_data_clean = hr_data_arr[outlier_mask]

# Interpolate
hr_resampled = np.interp(hr_time_arr, hr_time_clean, hr_data_clean)
print(hr_resampled)

# Summary

There are many techniques at our disposal in order to make real-world data conform to these assumptions. However, it will be important for you to consider the implications of applying these techniques to your data. If you preprocess your data too heavily, you may remove important information that will be useful down the road for other analyses.